# 검색 증강 생성

[검색 증강 생성(Retrieval Augmented Generation)](https://arxiv.org/abs/2005.11401)은 검색 기반 모델과 생성 모델을 결합하여 자연어 생성을 향상시키는 과정입니다. 이 과정에서는 관련 정보를 검색하고 이를 생성 프로세스에 통합합니다.

이 실습에서는 사용자가 다양한 와인에 대해 질문할 수 있는 RAG 애플리케이션 코드를 작성할 것입니다. 이를 통해 사용자는 구매 결정을 내리는 데 도움을 받을 수 있습니다. OpenSearch의 의미론적 검색(벡터 검색) 기능을 사용하여 가장 잘 일치하는 와인 리뷰를 검색하고, 이를 LLM(Large Language Model)에 제공하여 사용자의 질문에 답변할 것입니다.

## 1. 실습 전 준비사항

#### a. Python 의존성 다운로드 및 설치

이 노트북을 위해 몇 가지 라이브러리가 필요합니다. OpenSearch와 SageMaker를 위한 Python 클라이언트를 사용할 것이며, 텍스트 임베딩을 생성하기 위해 OpenSearch ML 클라이언트 라이브러리를 사용할 것입니다.

In [ ]:
!pip install opensearch-py-ml accelerate tqdm --quiet
!pip install setuptools==70.1.1 --quiet
!pip install sagemaker --upgrade --quiet
!pip install requests_aws4auth --quiet
!pip install alive-progress --quiet
!pip install deprecated --quiet


#OpenSearch Python SDK
!pip install opensearch_py  --quiet
#Progress bar for for loop
!pip install alive-progress  --quiet

# The version should already be 1.13.1 or higher. If not, we will restart the kernel.
# 버전은 이미 1.13.1 이상이어야 합니다. 그렇지 않은 경우 커널을 재시작합니다.

import torch
pytorch_version = torch.__version__
print( f"Pytorch version: {pytorch_version}")

def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
if pytorch_version.startswith('1.1'):
    from IPython.display import display_html
    restartkernel()
    
##You may safely ignore pip dependencies errors

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.32.101 requires botocore==1.34.101, but you have botocore 1.34.143 which is incompatible.
Pytorch version: 2.1.0


#### b. 라이브러리 가져오기 & 리소스 정보 초기화

아래 줄은 이 노트북에서 사용되는 모든 관련 라이브러리와 모듈을 가져옵니다.

In [ ]:
import boto3
import os
import time
import json
import pandas as pd
from tqdm import tqdm
import sagemaker
from opensearchpy import OpenSearch, RequestsHttpConnection
from sagemaker import get_execution_role
import random 
import string
import s3fs
from urllib.parse import urlparse
from IPython.display import display, HTML
from alive_progress import alive_bar
from opensearch_py_ml.ml_commons import MLCommonClient
from requests_aws4auth import AWS4Auth
import requests 

#### c. CloudFormation 스택 출력 변수 가져오기

우리는 계정에 CloudFormation 스택을 생성하여 몇 가지 리소스를 미리 구성해 두었습니다. 이 리소스들의 이름과 ARN은 이 실습 내에서 사용될 것입니다. 여기서 우리는 이러한 정보 변수들 중 일부를 로드할 것입니다.

In [ ]:
# Boto3 세션 만들기
session = boto3.Session()

# Account ID 가져오기
account_id = boto3.client('sts').get_caller_identity().get('Account')

# 현재 region 가져오기
region = session.region_name

cfn = boto3.client('cloudformation')

# Cloudformation 스택에서 출력 변수를 가져오는 메서드입니다. 
def get_cfn_outputs(stackname):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)['Stacks'][0]['Outputs']:
        outputs[output['OutputKey']] = output['OutputValue']
    return outputs

## 나머지 데모에 사용할 변수를 설정합니다.
cloudformation_stack_name = "genai-data-foundation-workshop"

outputs = get_cfn_outputs(cloudformation_stack_name)
aos_host = outputs['OpenSearchDomainEndpoint']
s3_bucket = outputs['s3BucketTraining']
bedrock_inf_iam_role = outputs['BedrockBatchInferenceRole']
bedrock_inf_iam_role_arn = outputs['BedrockBatchInferenceRoleArn']
sagemaker_notebook_url = outputs['SageMakerNotebookURL']

# 필요한 경우 쉽게 복사할 수 있도록 모든 변수를 출력합니다.
outputs

## 3. 데이터 준비
아래는 와인 리뷰 데이터셋을 로드하는 코드입니다. 이 데이터셋을 사용하여 사용자가 제공한 설명과 유사한 와인을 추천할 것입니다.

#### OpenSearch에 빠르게 로드하기 위해 레코드의 일부 샘플링
데이터가 129,000개의 레코드로 구성되어 있기 때문에, 이들을 벡터로 변환하고 벡터 저장소에 로드하는 데 시간이 걸릴 수 있습니다. 따라서 우리는 데이터의 일부(300개 레코드)만을 사용할 것입니다. 레코드의 인덱스에 해당하는 record_id라는 변수를 추가할 것입니다.

In [ ]:
url = "https://raw.githubusercontent.com/davestroud/Wine/master/winemag-data-130k-v2.json"
df = pd.read_json(url)
df_sample = df.sample(300,random_state=37).reset_index()
df_sample['record_id'] = range(1, len(df_sample) + 1)
df_sample[:5]

## 4. Amazon OpenSearch Service 도메인과 연결 생성
다음으로, Python API를 사용하여 OpenSearch 도메인과의 연결을 설정할 것입니다.

#### Secrets Manager에서 자격 증명 검색
코드에 사용자 이름과 비밀번호를 하드코딩하는 것을 피하기 위해, 우리는 클러스터를 배포할 때 사용자 이름과 비밀번호를 동적으로 생성했습니다. 이 사용자 이름과 비밀번호는 AWS Secrets Manager 서비스에 저장되어 있습니다. OpenSearch 연결을 설정하기 위해 Secrets Manager에서 비밀을 검색할 것입니다.

In [ ]:
kms = boto3.client('secretsmanager')
aos_credentials = json.loads(kms.get_secret_value(SecretId=outputs['DBSecret'])['SecretString'])

# 이 실습에서는 AWS Secrets 관리자 서비스에서 이미 생성한 자격 증명을 사용하겠습니다. Secrets
# 관리자 서비스를 사용하면 비밀을 안전하게 저장하고 코드를 통해 안전하게 검색할 수 있습니다.

auth = (aos_credentials['username'], aos_credentials['password'])

aos_client = OpenSearch(
    hosts = [{'host': aos_host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

## 5. Amazon Bedrock Titan Text 임베딩을 사용하여 텍스트를 벡터로 변환
Amazon Bedrock 서비스는 텍스트에 대한 벡터 임베딩을 생성하는 Amazon Titan Text 임베딩 v2 모델을 제공합니다. 이 모델을 우리의 주요 임베딩 모델로 사용할 것입니다.

#### Amazon Bedrock의 Titan Text 임베딩 모델을 호출하기 위한 헬퍼 메서드
Amazon Titan Text v2 임베딩 모델을 호출하기 위한 Python 헬퍼 메서드를 생성합니다. 우리는 `df_sample` 데이터 프레임을 업데이트하고 여기에 `embedding`이라는 새로운 열을 추가할 것입니다. 이 셀이 실행되면, 우리의 데이터 프레임은 OpenSearch에 로드될 준비가 됩니다. 실행을 완료하는 데 몇 분 정도 걸릴 수 있습니다.

In [ ]:
import boto3
import pandas as pd
import os
from typing import Optional

# 외부 종속성:
import boto3
from botocore.config import Config


bedrock_client = boto3.client(
    "bedrock-runtime", 
    region, 
    endpoint_url=f"https://bedrock-runtime.{region}.amazonaws.com"
)


def add_embeddings_to_df(df, text_column):

    # 임베딩을 저장할 빈 목록 만들기
    embeddings = []

    # 지정된 열의 텍스트를 반복합니다.
    for text in df[text_column]:
        embedding = embed_phrase(text)
        embeddings.append(embedding)
        
    # 임베딩을 데이터 프레임에 새 열로 추가합니다.
    df['embedding'] = embeddings

    return df

def embed_phrase( text ):
        
    model_id = "amazon.titan-embed-text-v2:0"  # 
    accept = "application/json"
    contentType = "application/json"

    # 요청 페이로드 준비
    request_payload = json.dumps({"inputText": text})


    response = bedrock_client.invoke_model(body=request_payload, modelId=model_id, accept=accept, contentType=contentType)

    # 응답에서 임베딩 추출
    response_body = json.loads(response.get('body').read())


    # 목록에 임베딩을 추가합니다.
    embedding = response_body.get("embedding")
    return embedding

df_sample = add_embeddings_to_df(df_sample, 'description')

df_sample[:5]

#### 간단한 입력 텍스트의 임베딩을 생성해 봅시다
이것이 부동소수점 숫자의 배열임을 볼 수 있습니다. 인간의 눈/뇌로는 이해하기 어렵지만, 이 숫자 배열은 텍스트의 의미와 지식을 포착하며, 이후 두 개의 다른 텍스트 블록을 비교하는 데 사용될 수 있습니다. 이 방법은 우리의 쿼리를 벡터 표현으로 변환하는 데 사용될 것입니다.

In [ ]:
## 입력 텍스트에 대한 벡터 임베딩 만들기
#칠면조 가슴살과 잘 어울리는 와인은?
input_text = "A wine that pairs well with turkey breast?"   

embedding = embed_phrase(input_text)

#텍스트 출력 및 임베딩
print(f"{input_text=}")

#1024 차원 벡터의 처음 10개 차원만 인쇄합니다. 
print(f"{embedding[:10]=}")

## 7. Amazon OpenSearch Service에 인덱스 생성
이전에 2-3개의 필드로 인덱스를 생성했던 것과 달리, 이번에는 여러 필드로 인덱스를 정의할 것입니다: `description` 필드의 벡터화와 데이터셋 내의 다른 모든 필드들을 포함합니다.

인덱스를 생성하기 위해, 먼저 JSON으로 인덱스를 정의한 다음, 앞서 초기화한 aos_client 연결을 사용하여 OpenSearch에 인덱스를 생성합니다.

In [ ]:
knn_index = {
    "settings": {
        "index.knn": True,
        "index.knn.space_type": "cosinesimil",
        "analysis": {
          "analyzer": {
            "default": {
              "type": "standard",
              "stopwords": "_english_"
            }
          }
        }
    },
    "mappings": {
        "properties": {
            "description_vector": {
                "type": "knn_vector",
                "dimension": 1024,
                "store": True
            },
            "description": {
                "type": "text",
                "store": True
            },
            "designation": {
                "type": "text",
                "store": True
            },
            "variety": {
                "type": "text",
                "store": True
            },
            "country": {
                "type": "text",
                "store": True
            },
            "winery": {
                "type": "text",
                "store": True
            },
            "points": {
                "type": "integer",
                "store": True
            },
            "wine_name": {
                "type": "text",
                "store": True
            },
        }
    }
}


위의 인덱스 정의를 사용하여 이제 Amazon OpenSearch에 인덱스를 생성해야 합니다. 이 셀을 실행하면 이미 이 노트북을 실행한 경우에도 인덱스를 다시 생성합니다.

In [ ]:
index_name = "wine_knowledge_base"

try:
    aos_client.indices.delete(index=index_name)
    print("Recreating index '" + index_name + "' on cluster.")
    aos_client.indices.create(index=index_name,body=knn_index,ignore=400)
except:
    print("Index '" + index_name + "' not found. Creating index on cluster.")
    aos_client.indices.create(index=index_name,body=knn_index,ignore=400)


생성된 인덱스 정보를 확인해 봅시다.

In [ ]:
aos_client.indices.get(index=index_name)

## 8. 원본 데이터를 인덱스에 로드
다음으로, 방금 생성한 인덱스에 와인 리뷰 데이터와 임베딩을 로드하겠습니다. 우리의 임베딩을 `description_vector` 필드에 저장할 것임을 주목하세요. 이 필드는 나중에 KNN 검색에 사용될 것입니다.

In [ ]:
for index, record in tqdm(df_sample.iterrows()):
    body={"description_vector": record['embedding'], 
           "description": record["description"],
           "points":record["points"],
           "variety":record["variety"],
           "country":record["country"],
           "designation":record["designation"],
           "winery":record["winery"],
          "wine_name":record["title"]
         }
    aos_client.index(index=index_name, body=body)
print("Records loaded...")

로드를 검증하기 위해, 인덱스 내의 문서 수를 조회해 보겠습니다. 인덱스에는 300개의 히트(또는 이전 샘플링에서 지정한 만큼의 수)가 있어야 합니다.

In [ ]:
res = aos_client.search(index=index_name, body={"query": {"match_all": {}}})
print("Records found: %d." % res['hits']['total']['value'])

## 9. "의미론적 검색(Semantic Search)"으로 벡터 검색

이제 요청된 설명과 가장 잘 일치하는 와인 리뷰를 찾기 위한 검색 쿼리를 실행하는 헬퍼 함수를 정의할 수 있습니다. `retrieve_opensearch_with_semantic_search` 함수는 검색 문구를 임베딩하고, 가장 가까운 벡터와 일치하는 인덱스를 검색한 후, 상위 결과를 반환합니다.

In [ ]:
def retrieve_opensearch_with_semantic_search(phrase, n=3):
    search_vector = embed_phrase(phrase)
    osquery={
        "_source": {
            "exclude": [ "description_vector" ]
        },
        
      "size": n,
      "query": {
        "knn": {
          "description_vector": {
            "vector":search_vector,
            "k":n
          }
        }
      }
    }

    res = aos_client.search(index=index_name, 
                           body=osquery,
                           stored_fields=["description","winery","points", "designation", "country"],
                           explain = True)
    top_result = res['hits']['hits']
    
    results = []
    
    for entry in top_result:
        result = {
            "description":entry['_source']['description'],
            "winery":entry['_source']['winery'],
            "points":entry['_source']['points'],
            "designation":entry['_source']['designation'],
            "country":entry['_source']['country'],
            "variety":entry['_source']['variety'],
            "wine_name":entry['_source']['wine_name'],
        }
        results.append(result)
    
    return results


샘플 질문을 사용하여 의미론적 검색으로 유사한 레코드를 가져옵니다.

In [ ]:
#스테이크와 잘 어울리는 최고의 호주 와인은?
question_on_wine="Best Australian wine that goes great with steak?"     

example_request = retrieve_opensearch_with_semantic_search(question_on_wine)
print(json.dumps(example_request, indent=4))

## 10. Amazon Bedrock - Anthropic Claude Sonnet 모델을 호출하는 메서드 준비

이제 사용자의 질문에 답변하기 위해 LLM을 호출하는 함수를 정의할 것입니다. LLM은 일반적인 목적의 데이터로 훈련되었기 때문에, 여러분의 와인 리뷰 지식을 가지고 있지 않을 수 있습니다.<br>
답변할 수 있을지라도, 그것이 여러분의 비즈니스가 선호하는 답변이 아닐 수 있습니다.<br>
예를 들어, `여러분의 경우에는 재고가 없는 와인을 추천하지 않기를 원할 것입니다.`

따라서 추천은 여러분의 컬렉션, <u>`즉 우리가 로드한 300개의 리뷰된 와인들 중 하나여야 합니다.`</u>

이 함수를 정의한 후, 우리는 와인 리뷰 데이터 없이 LLM이 질문에 어떻게 답변하는지 보기 위해 이를 호출할 것입니다.

In [ ]:
def query_llm_endpoint_with_json_payload(encoded_json):

    # Bedrock Runtime 클라이언트 생성
    bedrock_client = boto3.client('bedrock-runtime')

    # Claude 3 Sonnet의 모델 ID 설정하기
    model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
    accept = 'application/json'
    content_type = 'application/json'


    try:
        # 기본 요청 페이로드로 모델 호출하기
        response = bedrock_client.invoke_model(
            modelId=model_id,
            body=str.encode(str(encoded_json)),
            accept = accept,
            contentType=content_type
        )

        # 응답 본문 디코딩
        response_body = json.loads(response.get('body').read())
        return response_body
    except Exception as e:
        print(f"Error: {e}")
        return none

def query_llm(system, user_question):
    # 모델의 페이로드 준비
    payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10000,
        "system": system,
        "messages": [
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": f"{user_question}"
                }
              ]
            }
          ]
        })
    


    query_response = query_llm_endpoint_with_json_payload(payload)

    return query_response['content'][0]['text']


와인 추천에 대해 생성된 결과를 확인해 봅시다. 이는 우리가 재고로 가지고 있는 와인 중 하나가 아닐 수 있습니다.

In [ ]:
def query_llm_without_rag(question):
    
    #Claude 모델에는 두 부분으로 구성된 프롬프트가 있습니다. 
    #시스템 프롬프트는 모델에 어떤 역할을 수행할지 안내합니다.
    system_prompt = (
                    #소믈리에는 와인에 대한 방대한 지식을 바탕으로 사람들이 즐길 수 있는 훌륭한 추천을 하는 사람입니다.
                    #그리고 와인 이름은 영문으로, 와인 설명은 한글로 답해주세요.
                    f"You are a sommelier that uses their vast knowledge of wine to make great recommendations people will enjoy." 
                    f"And please answer the wine name in English and the wine description in Korean."
    )
    
    #사용자 프롬프트는 모델이 질문에 답하기 위해 참조해야 하는 컨텍스트가 있는 엔지니어 프롬프트입니다.
    user_prompt = (
        #소믈리에는 와인 품종과 원산지를 반드시 기재해야 합니다,
        #그리고 고객 질문과 관련된 다채로운 설명을 제공합니다.
        #고객 질문입니다: {question}
        #답변 마지막에 와인 이름을 새 줄에 와인 이름 형식으로 입력하세요: <wine name>        
        f" As a sommelier, you must include the wine variety, the country of origin, "                                           
        f"and a colorful description relating to the customer question."                                                           
        f"\n Customer question: {question}"                                                                                        
        f"\n Please provide name of the wine at the end of the answer, in a new line, in format Wine name: <wine name>"            
    )
    return query_llm(system_prompt, user_prompt)


#스테이크와 잘 어울리는 최고의 호주 와인은?
question_on_wine="Best Australian wine that goes great with steak?"     

#RAG가 없는 LLM의 추천 와인: {query_llm_without_rag(question_on_와인)}
print(f"The recommened wine from LLM without RAG: \n{query_llm_without_rag(question_on_wine)}\n")

#### 환각(hallucination) 테스트
마지막 줄에서 와인 이름을 복사하여 아래의 질문 변수에 붙여넣어, 우리 재고에 이 와인이 있는지 확인해 봅시다. 반환된 와인 목록을 검토해 주세요. 포르투갈 와인일 수는 있지만, 모델이 추천한 것과 정확히 일치하지 않을 수 있습니다. 이를 환각이라고 부릅니다. 모델이 일반적인 지식을 바탕으로 임의의 추천을 했기 때문입니다.

__참고:__ 위 모델이 추천한 와인 이름이 아래 `wine_name` 변수에 보이지 않는다면, 추천된 와인이 우리의 OpenSearch 인덱스에 없다는 것을 확인할 수 있도록 이를 교체해야 합니다.

In [ ]:
wine_name = "Penfolds Bin 389 Kalimna Shiraz"
example_request = retrieve_opensearch_with_semantic_search(wine_name)

#리뷰에서 와인 기록과 일치하는 와인
print("Matching wine records in our reviews:")      
print(json.dumps(example_request, indent=4))

## 11. 검색 증강 생성(Retrieval Augmented Generation)
---
LLM의 환각 문제를 해결하기 위해, 우리는 LLM에 더 많은 컨텍스트를 제공할 수 있습니다. 이를 통해 LLM은 컨텍스트 정보를 사용하여 모델을 미세 조정하고 사실에 기반한 결과를 생성할 수 있습니다. RAG는 LLM 환각 문제에 대한 해결책 중 하나입니다.

#### OpenSearch 검색 결과를 사용하여 LLM을 위한 프롬프트 생성 (RAG)

우리는 Anthropic Claude Sonnet 3 모델을 원샷 프롬프팅 기법과 함께 사용할 것입니다. 모델에 대한 지시사항 내에서, 우리는 샘플 와인 리뷰와 모델이 사용자의 질문에 답변하는 방법을 제공할 것입니다. 프롬프트의 끝에는 OpenSearch에서 검색한 와인 리뷰가 모델이 사용할 수 있도록 포함될 것입니다.

모델에 쿼리하기 전에, 아래의 `generate_rag_based_system_prompt` 함수를 사용하여 사용자 프롬프트를 구성합니다. 이 함수는 OpenSearch 클러스터에서 일치하는 와인을 검색하기 위한 입력 문자열을 받아, LLM을 위한 사용자 프롬프트를 작성합니다.

시스템 프롬프트는 LLM이 수행할 역할을 정의합니다.

사용자 프롬프트는 LLM 모델이 사용자의 질문에 답변하기 위해 사용하는 지시사항과 컨텍스트 정보를 포함합니다.

프롬프트는 다음과 같은 형식을 가집니다:

**시스템 프롬프트:**

```
당신은 와인에 대한 광범위한 지식을 사용하여 사람들이 즐길 수 있는 훌륭한 추천을 하는 소믈리에입니다.
```

**사용자 프롬프트**
```
소믈리에로서, 당신은 와인 품종, 원산지 국가, 그리고 사용자의 질문과 관련된 생생한 설명을 포함해야 합니다.

Data:{'description': '이 향기로운 화이트 와인은 강렬하고 크리미한 층의 핵과류와 바닐라 맛으로 춤추며, 처음부터 끝까지 활기차고 균형 잡힌 맛을 유지합니다. 풍부한 과일은 나파 밸리의 비교적 서늘한 오크 놀 섹션에서 재배됩니다. 시간이 지나고 글라스에서 더욱 발전할 것입니다.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}

Recommendation: 당신을 위한 훌륭한 와인이 있습니다. 미국 나파 밸리의 오크 놀 섹션에 있는 Darioush 와이너리의 드라이하고 중간 바디감의 화이트 와인입니다. 바닐라와 오크 풍미가 있습니다. 와인 스펙테이터에서 92점을 받았습니다.

Data: {retrieved_documents}

사용자의 질문 그대로
```

### 프롬프트 패키징 및 LLM 쿼리
프롬프트로 LLM을 쿼리하기 위한 최종 함수를 만들 것입니다. `query_llm_with_rag`는 RAG에서 LLM을 호출하는 함수입니다.

`query_llm_with_rag`는 이 모듈에서 우리가 수행한 모든 것을 결합합니다. 다음과 같은 모든 작업을 수행합니다:
- "description vector"를 사용하여 의미론적 검색으로 OpenSearch 인덱스에서 관련 와인을 검색합니다.
- 검색 결과로부터 LLM 프롬프트를 생성합니다.
- RAG를 사용하여 LLM에 응답을 쿼리합니다.

In [ ]:
def query_llm_with_rag(user_question):
    retrieved_documents = retrieve_opensearch_with_semantic_search(user_question)
    #"{'description': '이 향기로운 화이트 와인은 강렬하고 크리미한 층의 핵과류와 바닐라 맛으로 춤을 추며, 처음부터 끝까지 활기차고 균형 잡힌 맛을 유지합니다. 풍부한 과일은 나파 밸리의 비교적 서늘한 오크 놀 섹션에서 재배됩니다. 이 와인은 시간이 지나고 글라스에서 더욱 발전할 것입니다.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}"
    one_shot_description_example = "{'description': 'This perfumey white dances in intense and creamy layers of stone fruit and vanilla, remaining vibrant and balanced from start to finish. The generous fruit is grown in the relatively cooler Oak Knoll section of the Napa Valley. This should develop further over time and in the glass.', 'winery': 'Darioush', 'points': 92, 'designation': None, 'country': 'US'}"

    #"당신을 위한 훌륭한 와인이 있습니다. 미국 나파 밸리의 오크 놀 섹션에 있는 Darioush 와이너리의 드라이하고 중간 바디감의 화이트 와인입니다. 바닐라와 오크 풍미가 있습니다. 와인 스펙테이터에서 92점을 받았습니다."
    one_shot_response_example = "I have a wonderful wine for you. It's a dry, medium bodied white wine from Darioush winery in the Oak Knoll section of Napa Valley, US. It has flavors of vanilla and oak. It scored 92 points in wine spectator."
    system_prompt= (
        #와인에 대한 방대한 지식을 활용하여 사람들이 즐길 수 있는 훌륭한 추천을 하는 소믈리에입니다.
        #그리고 와인 이름은 영문으로, 와인 설명은 한글로 답해주세요.
        f"You are a sommelier that uses vast knowledge of wine to make great recommendations people will enjoy"                                
        f"And please answer the wine name in English and the wine description in Korean."

    )
    user_prompt = (
        #소믈리에는 와인 품종, 원산지 및 사용자 질문과 관련된 다채로운 설명을 포함해야 합니다. '와인 데이터' 섹션에서만 고객 질문과 가장 잘 어울리는 와인을 선택해야 합니다. 제공된 와인 데이터 외에 다른 와인을 제안하지 마세요. 고객 질문에 가장 적합하지 않다고 해서 반드시 최고 등급의 와인을 선택할 필요는 없습니다.
        f"As a sommelier, you must include the wine variety, the country of origin, and a colorful description relating to the user question. You are must pick a wine in \"Wine data\" section only, one that matches best the customer question. Do not suggest anything outside of the wine data provided. You don't necessarily have to pick the top rated wine if its not best suited for customer question.\n"
        f"Wine data: {one_shot_description_example} \n Recommendation: {one_shot_response_example} \n"
        f"Wine data: {retrieved_documents} \n"
        f"Customer Question: {user_question} \n"        
    )
    response = query_llm(system_prompt, user_prompt)
    return response

#### 마지막으로 함수를 호출하여 와인 추천을 받아보겠습니다.

In [ ]:
#스테이크와 잘 어울리는 최고의 호주 와인은?
question_on_wine="Best Australian wine that goes great with steak?"     
recommendation = query_llm_with_rag(question_on_wine)
print(recommendation)

#위의 권장 사항에 대해 검색된 문서는 다음과 같습니다.
print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_opensearch_with_semantic_search(question_on_wine), indent=4)}")     

#### 이탈리아 와인으로 변경해 보겠습니다. 일치하는 결과가 나올 것입니다.
동일한 메서드를 다시 호출하여 카탈로그에 이탈리아 와인이 있는지 확인하겠습니다.

In [ ]:
#스테이크와 잘 어울리는 최고의 이탈리아 와인은?
question_on_wine="Best Italian wine that goes great with steak?"        
recommendation = query_llm_with_rag(question_on_wine)
print(recommendation)

#위의 권장 사항에 대해 검색된 문서는 다음과 같습니다.
print(f"\n\ndocuments retrieved for above recommendations were \n\n{json.dumps(retrieve_opensearch_with_semantic_search(question_on_wine), indent=4)}")     

스테이크와 잘 어울리는 호주 와인을 요청했는데 우리 컬렉션에는 그런 와인이 없다는 것을 눈치채셨을 것입니다. <br>
따라서 모델은 정중하게 다음과 같이 변명합니다.<br> 
질문을 바꿔서 LLM이 셀렉트 리스트에서 귀하의 질문에 가장 잘 맞는 와인을 추천하는 방법을 확인할 수 있습니다.

### 결론
이 실습에서는 간단한 와인 추천 챗봇을 구축했습니다. 이 실습에서는 데이터에 대한 벡터 임베딩을 생성하기 위해 Amazon Bedrock titan v2 모델을 사용했습니다. 그런 다음 이 데이터를 `description_vector` 필드를 사용하여 OpenSearch 인덱스에 로드했습니다. 검색 시에는 Amazon Titan v2 모델을 다시 사용하여 쿼리 질문을 벡터 임베딩으로 변환하고 시맨틱 검색을 사용하여 결과를 검색했습니다. 그런 다음 이 결과를 Anthropic Claude Sonnet 3 모델에 전달하여 카탈로그 내에서 와인을 추천할 수 있었습니다.

## 실습 완료 - 이제 실습 지침 섹션으로 돌아갈 수 있습니다.